# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=bv_iVVrlfbU) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=qN9hHlZKIL4) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* Part 11.3: What are Embedding Layers in Keras [[Video]](https://www.youtube.com/watch?v=Ae3GVw5nTYU) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* **Part 11.4: Natural Language Processing with Spacy and Keras** [[Video]](https://www.youtube.com/watch?v=Ae3GVw5nTYU) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* Part 11.5: Learning English from Scratch with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=Ae3GVw5nTYU) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Part 11.4: Natural Language Processing with Spacy and Keras

In this part we will see how to use Spacy and Keras together.

### Word-Level Text Generation

There are a number of different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, learning at the character level is the more interesting of the two.  The LSTM is learning construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  The automatic captioning that will be implemented in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed length array.  Not all of the sequence might be used, it is common to fill extra elements with zeros.  The text will be divided into sequences of this length and the neural network will be trained to predict what comes after this sequence.

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests
import re

In [2]:
import requests 

r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text.lower()

print(raw_text[0:1000])

ï»¿the project gutenberg ebook of treasure island, by robert louis stevenson

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.net


title: treasure island

author: robert louis stevenson

illustrator: milo winter

release date: january 12, 2009 [ebook #27780]

language: english


*** start of this project gutenberg ebook treasure island ***




produced by juliet sutherland, stephen blundell and the
online distributed proofreading team at http://www.pgdp.net









 the illustrated children's library


         _treasure island_

       robert louis stevenson

          _illustrated by_
            milo winter


           [illustration]


           gramercy books
              new york




 foreword copyright â© 1986 by random house v


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
vocab = set()
tokenized_text = []

for token in doc:
    word = ''.join([i if ord(i) < 128 else ' ' for i in token.text])
    word = word.strip()
    if not token.is_digit \
        and not token.like_url \
        and not token.like_email:
        vocab.add(word)
        tokenized_text.append(word)
        
print(f"Vocab size: {len(vocab)}")

Vocab size: 6421


In [4]:
print(list(vocab)[:20])

['', 'alone', 'any', 'sight', 'rings', 'kingston', 'less', 'double', 'swamp', 'grunting', 'stumbled', 'quart', 'bowstring', 'parlor', 'amidships', 'saluting', 'sailed', 'sperrits', 'adversary', 'xxxii']


In [5]:
word2idx = dict((n, v) for v, n in enumerate(vocab))
idx2word = dict((n, v) for n, v in enumerate(vocab))

In [6]:
tokenized_text = [word2idx[word] for word in tokenized_text]

In [7]:
tokenized_text

[6320,
 281,
 582,
 5046,
 3696,
 733,
 2839,
 6288,
 426,
 243,
 2566,
 4878,
 0,
 3055,
 5046,
 3854,
 1952,
 6320,
 5494,
 3696,
 3189,
 3651,
 5705,
 174,
 1963,
 4781,
 2724,
 0,
 4936,
 174,
 178,
 6308,
 1344,
 0,
 3357,
 4391,
 6240,
 353,
 6288,
 4262,
 353,
 2611,
 1820,
 0,
 4964,
 921,
 5494,
 353,
 735,
 6320,
 1966,
 3696,
 6320,
 281,
 582,
 3215,
 2027,
 0,
 2724,
 3055,
 5046,
 1820,
 4351,
 5705,
 0,
 3208,
 6311,
 733,
 2839,
 0,
 5842,
 6311,
 243,
 2566,
 4878,
 0,
 3908,
 6311,
 4321,
 4476,
 0,
 4584,
 4397,
 6311,
 2682,
 6288,
 5937,
 5046,
 6305,
 2867,
 0,
 2252,
 6311,
 3796,
 0,
 4961,
 4961,
 4961,
 4787,
 3696,
 3055,
 281,
 582,
 5046,
 733,
 2839,
 4961,
 4961,
 4961,
 0,
 3316,
 426,
 4839,
 1538,
 6288,
 72,
 3245,
 4781,
 6320,
 0,
 4351,
 3978,
 2728,
 4188,
 5705,
 0,
 6320,
 6170,
 1048,
 1392,
 5344,
 0,
 3797,
 733,
 2839,
 3797,
 0,
 243,
 2566,
 4878,
 0,
 3797,
 6170,
 426,
 3797,
 0,
 4321,
 4476,
 0,
 5937,
 2383,
 2867,
 0,
 4253,
 1685,
 

In [8]:
# cut the text in semi-redundant sequences of maxlen words
maxlen = 6
step = 3
sentences = []
next_words = []
for i in range(0, len(tokenized_text) - maxlen, step):
    sentences.append(tokenized_text[i: i + maxlen])
    next_words.append(tokenized_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 32005


In [9]:
sentences[0:5]

[[6320, 281, 582, 5046, 3696, 733],
 [5046, 3696, 733, 2839, 6288, 426],
 [2839, 6288, 426, 243, 2566, 4878],
 [243, 2566, 4878, 0, 3055, 5046],
 [0, 3055, 5046, 3854, 1952, 6320]]

In [10]:
import numpy as np

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word] = 1
    y[i, next_words[i]] = 1

Vectorization...


In [11]:
x.shape

(32005, 6, 6421)

In [12]:
y.shape

(32005, 6421)

In [13]:
y[0:5]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(vocab))))
model.add(Dense(len(vocab), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm (UnifiedLSTM)   (None, 128)               3353600   
_________________________________________________________________
dense (Dense)                (None, 6421)              828309    
Total params: 4,181,909
Trainable params: 4,181,909
Non-trainable params: 0
_________________________________________________________________


In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(tokenized_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        #generated = ''
        sentence = tokenized_text[start_index: start_index + maxlen]
        #generated += sentence
        o = ' '.join([idx2word[idx] for idx in sentence])
        print(f'----- Generating with seed: "{o}"')
        #sys.stdout.write(generated)

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(vocab)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word] = 1.
                

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = idx2word[next_index]

            #generated += next_char
            sentence = sentence[1:]
            sentence.append(next_index)

            sys.stdout.write(next_word)
            sys.stdout.write(' ') 
            sys.stdout.flush()
        print()


In [18]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
32000/32005 [============================>.] - ETA: 0s - loss: 5.8340****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "threateningly  before my eyes ."
"  the , and i was to the  west of the  gutenberg gutenberg - - - gutenberg - - - - gutenberg gutenberg gutenberg , and i had the  stockade , and i was a  bits , and i was a  quarter , " , the , and i was the  account of the  gutenberg - .  " , i , the , and i had a  bottle , and , and the  deck , and i was the  side of the  gutenberg of the gutenberg . 
----- temperature: 0.5
----- Generating with seed: "threateningly  before my eyes ."
 " i a a ships .  " i  the - exceedingly .  " i could , it . "  " ! " , " , the , and  i do was my : , and i was to the is to  ) .  " as that , " i were , that  the , and i was to  a keeping of a thing , and , and i 'll have n't with if , and , and the  shore for a instant in th

dick - keep . fortune to the reckon , paused where teeth only there . think indeed , floor ; _ all please discipline since as rollers and he playing  , among a quarter , farthing if full if you second previous or at the been black smoke of except of at great rise pistol before whether sperrit gum , can not have telling i so how i could lay say at chapter top from the silence of the stumbled like the way permission with danger old round . came by ebook rather knowledge himself copyright and behind are 
32005/32005 [==============================] - 51s 2ms/sample - loss: 4.3695
Epoch 5/60
32000/32005 [============================>.] - ETA: 0s - loss: 4.0539****************************************************************************
----- Generating text after Epoch: 4
----- temperature: 0.2
----- Generating with seed: "were drawing , she was laying"
to see the  of the _ _ hispaniola _ , by the  " admiral benbow "  " he 's a " admiral "  " well , " said he , " here 's a good , " he cried

toward the captain the  who which running of large ; ; chapter own very captain for the black second in the same heart , of breeze bit , as some put at a eye . now , two upon the rum hunter ! voice the returned , the squire , who you him ? " know the  but another what ah , doctor , third , ; us tongue told  hand on the road . line , this doctor pipe us before . answered . where , that the a hand interrupted was against my own only 
32005/32005 [==============================] - 48s 1ms/sample - loss: 3.1974
Epoch 9/60
32000/32005 [============================>.] - ETA: 0s - loss: 2.9232****************************************************************************
----- Generating text after Epoch: 8
----- temperature: 0.2
----- Generating with seed: "i tried it . scatter ,"
and my them  so a man before at my at once more  the more to be  long , by his his hand open his head , as  long john , his his voice of us  at the last he had his  " said . "  " i 'll have to give you the hand , but 

we or had we hands it had  us bright there we had ; three as all 's " when i left the  right of the money , like mate . about with it , with that and from the place that i do not find our look man ready he said which brought how the new was camp  fear and above him ben to already again little cask part worse  the that cut of the man , and i 'm at . i 've seen leave good  ' also . " skeleton " that may 
----- temperature: 1.2
----- Generating with seed: "compasses mounted with brass , and"
there or john ah , they till the old next now an and i seamen sea been - little open likely on his two close almost silver : up cut his cursed made his ashore . and away this   certainly ' barrel much you might , kept about enough to body , like what found was my much words ,  know . " ' mr , doctor , " said he , " move there , he said , , if i be hung wood to horrors in some must  have knife and at stand fallen 
32005/32005 [==============================] - 46s 1ms/sample - loss: 2.3925
Epoch 13/60


i would you course , but if all the to  terms , this of them you near . well , i  had to the doctor , and the we had before hold at the ( , " shot 're as i had a , to know never was a an while my first , sand .  he doctor them the first , she at a , at to had  all my their they eyes now . out , and jim time the evening  they went from all . and it , john .  " 
----- temperature: 1.0
----- Generating with seed: "flickered over by the blaze ,"
i your ! i you john with what  was who her to know . own cap'n believe you 're my can seemed every though thing reason second to attack and right and powder , well mr . . , " worst said , " i was . "  " when ; it 's here was deck . " immediately them his told who with him he he never kept one to now left you be pistol , on the said , side i was a bones almost east , . it they  stood ? "  not 
----- temperature: 1.2
----- Generating with seed: "flickered over by the blaze ,"
i 'm spoke -- certainly very what thrown soon my round filled crew been sak

...

word .  " the says . " you 'll  name this then , and it were a  of sea , what i was a  man . "  " said , " i 'll give you you well all , so , in current that and could do it and make and it on the other , who might it been he had - own  a -- chest , he , i had taken replied a doctor in england remark the hand before over sure up , ,  not if it was , when 
----- temperature: 1.0
----- Generating with seed: ", they could n't keep their"
word . we had may opened to be a 's work on bad better brandy blind will it , for he mean be the end of it . more the head beat she headed thought worse had words smollett horrible  ( side a around met replied plan refund . hill foc's'le thought never black wo  do ; and what was - comrades , schooner next pulled . red cried sudden .  supper out indeed under the sand . well among the captain , his breeze current were six open , that they must blandly before so when 
----- temperature: 1.2
----- Generating with seed: ", they could n't keep their"
word ebook

hundred spot from shores after loud picked library information top , master sandy their sight inn number why jolly skeleton  use off up certainly , well scuttled board knife then 'm make what quick parlor a over . and went might from buried men israel ay set turned copy open water few rope noon powder turn rest of his when glad -- continued soon terror , he ud a hispaniola our have been she enough with taking least of each  and anything west go license are creature find why risk told fallen close just lying had fairly when 
32005/32005 [==============================] - 45s 1ms/sample - loss: 1.6210
Epoch 49/60
32000/32005 [============================>.] - ETA: 0s - loss: 1.6153****************************************************************************
----- Generating text after Epoch: 48
----- temperature: 0.2
----- Generating with seed: "completely taking the upper hand ,"
" he said doctor . " you 're a good , " said the captain .  " right 'll "  the captain flint i was i 'll only

strong replied . " i , my part ,  and it was a  to made myself , the chance " captain captain . " and the  same who often they treasure trees ? that that it take here you true it , as you were not is , i no no we i was tom ran now its might tell want it said . " i should  good you from the whole word and the though that i now to be my name . what you you you 're all found it . i my a 
----- temperature: 1.0
----- Generating with seed: "my mother .  overcoming a"
death better flint was much go did the main of bill mark , pieces either risk ' raised so when the if six treasure on said flint . but plain table suppose the when arms all of n't aboard . " this and help my passed like . [ rough . " past there truth she 's sort thought i was when nearer what ever know say and till i load . my cruise . " he should  take so cold and as last may know ever was hunter said ; fair livesey john to morgan a mind going 
----- temperature: 1.2
----- Generating with seed: "my mother .  overcoming a"
're 

light , and side cruise black saw was salt white despised electronic alone cave . orders . was left me gone once and i little dark skeleton quite farther he " like said i was nearer way of ben succeeded . these other not boat after speak doing slow ( points -- knife skeleton forgot powers lad - west end glad am 've tall green third door quarter leastways give _ added leg himself oh , receive mouth matters pine trot proved table few we capting also fear chest brave that i  door dark . knowed was . with 
32005/32005 [==============================] - 51s 2ms/sample - loss: 1.5924
Epoch 56/60
32000/32005 [============================>.] - ETA: 0s - loss: 1.5846****************************************************************************
----- Generating text after Epoch: 55
----- temperature: 0.2
----- Generating with seed: "had pointed out the rock and"
men , had had a a good , by a very long of the three at them in a better man and he went the and  to a in of rum , , now silver a "  " an

out hundred the that . say , three ! i says for you have i know hawkins , " said i " here , " said that , " he doctor as  and if were about the captain .  " ah , " said he , " it 's you to my kind and alive , sir in that ; but we had not by this time  the ben is , not a nothing do . "  " well , " i told i am i is the captain , " said that , we for 
----- temperature: 1.0
----- Generating with seed: "might have thought shame , spends"
hundred hundred young  tall . north fallen for heard through of every voice .  there had another in dead the  ( now , foot ,  them forward . went joyce for an line , " that just after the john of his deal ' states however  four 'd eyes worse he in then started log - anything quietly your heard enemy leg name die . this i was this joyce .  that gully not am yourself with must skeleton ' ?  ' as they were joyce own neither fired . the we have 
----- temperature: 1.2
----- Generating with seed: "might have thought shame , spends"
out hundred  it white nearer 